# Constraining the A.v. model main carbon metabolism 

There are particular constraints of A.v. that are currently not present in the model such as the useage of the Entner–Doudoroff Pathway (ED) and the pentose phosphate pathway (PPP) as well as sucrose metabolism particularly fructose catabolism. 

While we have MFA results (Wu et al. 2019) to constrain the model but we would like to use that data in the future to test the model so we will try to constrain flux to the ED and PP pathways with gene knockout first.  

The main supporting texts ###Finish###







In [1]:
import cobra.test
import pandas
import os
from os.path import join
from cobra import Model, Reaction, Metabolite
from cobra.io import save_json_model
from cobra.io import write_sbml_model
from cobra.io import read_sbml_model

import numpy as np
import pandas as pd




In [2]:
#add model
model = read_sbml_model("../Data/Models/iAA1300.xml")
model.medium


{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 0.05,
 'EX_mn2_e': 1000.0,
 'EX_fe2_e': 0.05,
 'EX_glc__D_e': 5.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 5.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_n2_e': 1000.0}

In [3]:
#Lets set gluose uptake to experimentally determined number 15 mmol (Wu et al) and make it diazotrophic by setting ammonia uptake to zero
model.reactions.get_by_id("EX_glc__D_e").lower_bound = -15
model.reactions.get_by_id("EX_glc__D_e").upper_bound = -15

model.reactions.get_by_id("EX_nh4_e").lower_bound = 0
model.reactions.get_by_id("EX_nh4_e").upper_bound = 0

solution = model.optimize()

#save to dataframe
df= pd.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('../Data/FBA_results/model_n2_glu_15.csv')


model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.005112,0,0.00%
cl_e,EX_cl_e,0.005112,0,0.00%
cobalt2_e,EX_cobalt2_e,2.455E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006963,0,0.00%
fe2_e,EX_fe2_e,0.008105,0,0.00%
fe3_e,EX_fe3_e,0.007668,0,0.00%
glc__D_e,EX_glc__D_e,15,6,100.00%
h_e,EX_h_e,1.225,0,0.00%
k_e,EX_k_e,0.1917,0,0.00%
mg2_e,EX_mg2_e,0.00852,0,0.00%


### Lets check out the model in Escher

In [9]:
import escher
from escher import Builder
from time import sleep

In [10]:
 Builder(model_json = "../Data/Models/iAA1300.json",
                 map_json = '../Data/Escher/Maps/central_metabolism.json',
                  reaction_data = solution.fluxes
                      )

Builder(reaction_data={'ALATA_D2': 0.0, 'SHCHD2': 0.0002190118504573613, 'CPPPGO': 0.0002190118504573613, 'GTH…

In [11]:
#Lets set gluose uptake to experimentally determined number ~15 mmol (Wu et al) and make uptake ammonia for non-diazotrophic growth
model.reactions.get_by_id("EX_glc__D_e").upper_bound = -15

model.reactions.get_by_id("EX_nh4_e").lower_bound = -1000
model.reactions.get_by_id("EX_nh4_e").upper_bound = 0

solution = model.optimize()

#save to dataframe
df= pd.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('../Data/FBA_results/model_nh4_glu_15.csv')


model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.007385,0,0.00%
cl_e,EX_cl_e,0.007385,0,0.00%
cobalt2_e,EX_cobalt2_e,3.547E-05,0,0.00%
cu2_e,EX_cu2_e,0.001006,0,0.00%
fe2_e,EX_fe2_e,0.02279,0,0.00%
glc__D_e,EX_glc__D_e,15,6,100.00%
k_e,EX_k_e,0.2769,0,0.00%
mg2_e,EX_mg2_e,0.01231,0,0.00%
mn2_e,EX_mn2_e,0.0009804,0,0.00%
mobd_e,EX_mobd_e,9.931E-06,0,0.00%


In [12]:
 Builder(model_json = "../Data/Models/iAA1300.json",
                 map_json = '../Data/Escher/Maps/central_metabolism.json',
                  reaction_data = solution.fluxes
                      )

Builder(reaction_data={'ALATA_D2': 0.0, 'SHCHD2': 0.00031638595490031754, 'CPPPGO': 0.00031638595490031754, 'G…

## As we can see the growth rate is far to fast for both diazotrophic and ammonia supplemented with a experimental glucose uptake rate. But we can see nitrogenase is working and can grow diazotrophically just a bit slower. As we still want to work on the carbon catabolism we will address the ED and PP pathways next

------------------------------

First we will look at the annotate PFK gene in A.v.:

While this gene Avin_23410 does have some homology to the E coli PFK2 enzyme it is also has high homology to 1-P fructose kinases and other genes in a simple blast search. We cannont be sure that this PFK is active in the forward direction making 1-6-fru in the EMP pathway. 

In [13]:
model.reactions.PFK

Reaction identifier,PFK
Name,Phosphofructokinase
Memory address,0x01bdb1ccc748
Stoichiometry,"atp_c + f6p_c --> adp_c + fdp_c + h_c ATP C10H12N5O13P3 + D-Fructose 6-phosphate --> ADP C10H12N5O10P2 + D-Fructose 1,6-bisphosphate + H+"
GPR,Avin_23410
Lower bound,0
Upper bound,0


In [14]:
model.reactions.get_by_id("EX_glc__D_e").lower_bound = -15
model.reactions.get_by_id("EX_glc__D_e").upper_bound = -15


model.reactions.get_by_id("EX_nh4_e").lower_bound = 0
model.reactions.get_by_id("EX_nh4_e").upper_bound = 0

#set pfk to zero
model.reactions.get_by_id("PFK").lower_bound = 0
model.reactions.get_by_id("PFK").upper_bound = 0

solution = model.optimize()

#save to dataframe
df= pd.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('../Data/FBA_results/model_n2_glu_15.csv')


model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.005112,0,0.00%
cl_e,EX_cl_e,0.005112,0,0.00%
cobalt2_e,EX_cobalt2_e,2.455E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006963,0,0.00%
fe2_e,EX_fe2_e,0.008105,0,0.00%
fe3_e,EX_fe3_e,0.007668,0,0.00%
glc__D_e,EX_glc__D_e,15,6,100.00%
h_e,EX_h_e,1.225,0,0.00%
k_e,EX_k_e,0.1917,0,0.00%
mg2_e,EX_mg2_e,0.00852,0,0.00%


In [15]:
 Builder(model_json = "../Data/Models/iAA1300.json",
                 map_json = '../Data/Escher/Maps/central_metabolism.json',
                  reaction_data = solution.fluxes
                      )

Builder(reaction_data={'ALATA_D2': 0.0, 'SHCHD2': 0.0002190118504573613, 'CPPPGO': 0.0002190118504573613, 'GTH…

#### While PFK deletion does stop flux from glycolosis we now have a weird pentose phosphate loop to make the g3p necessary. While there might be more ways to adjust the model to make sure it sends flux to the ED pathway I want to add as little subjective assumptions to this as possible. 

#### So we can start by adding constraints that were determined in the MFA study (Wu et al). We will do this by adding a dummy variable similar to the COBRA matlab function addRatioReaction and adapted from Yanfen Fu et al. (10.1128/mBio.00406-19) 


In [39]:
def ratioReaction(Model,rxn1,rxn2,ratioCoeff1,ratioCoeff2):
    """
    A function to add a dummy variable to a reactions stoichometery to force flux through both reactions at a specfic ratio
    
    
    
    The two reactions as "RXN1", "RXN2"
    
    The logic of the ratio is this: the rxn1 will produce the dummy variable at the coeeficent given by
    ratioCoeff1  while rxn2 must consume the dummy variable at the coefficent of ratioCoeff2 
        EX: Flux from pyruvate goes to pyruvate dehydrogenase  100x more than pyruvate carboxylase 
            So its a 0.01 ratio so the function will be written (model, PDH, PC, 1, 100)
            Where: 
            PDH - pyr + coa + nad -> accoa + nadh + co2 + DUMMY
            PC - pyr + atp + co2 + 100 DUMMY -> oaa + ADP + H + pi 
    """
  

    model = Model
    model_ratioed = model.copy()
    reaction1 = model_ratioed.reactions.get_by_id(rxn1)
    reaction2 = model_ratioed.reactions.get_by_id(rxn2)
    
    #create ghost metabolite and add to reactions
    
    ghost_id = "GM_{}_{}".format(rxn1, rxn2)
    ghost_metabolite = cobra.Metabolite(ghost_id, name = ghost_id, compartment = "c" )
    
    reaction1.add_metabolites({ghost_metabolite:ratioCoeff1})
    reaction2.add_metabolites({ghost_metabolite:(-1*ratioCoeff2)})
    
    #cobra.io.save_json_model(model, outfile)


    print("Growth rate of model without ratio: {}".format(model.optimize().objective_value))
    print("Growth rate of model with new ratio: {}".format(model_ratioed.optimize().objective_value))
    
    ratioed_fluxes = model_ratioed.optimize().fluxes

In [40]:
ratioReaction(model, "EDD", "GND", 18, 82)

Growth rate of model without ratio: 0.9821159213334587
Growth rate of model with new ratio: 0.9526447415857384


In [41]:
 
solution = model.optimize()
    
Builder(model_json = "../Data/Models/iAA1300.json",
                 map_json = '../Data/Escher/Maps/central_metabolism.json',
                  reaction_data = ratioed_fluxes
                      )

NameError: name 'ratioed_fluxes' is not defined

In [38]:
model.optimize().fluxes

ALATA_D2      0.000000
SHCHD2        0.000219
CPPPGO        0.000219
GTHOr         0.000000
GLYCTO2       0.000000
               ...    
QCCOR        82.464122
VNIT          0.000000
FENIT         0.000000
CYTCBB3pp     0.000000
NADTRHD       0.000000
Name: fluxes, Length: 2289, dtype: float64